In [6]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

In [7]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

labels = [
    "This comment expresses trust in the brand",
    "This comment expresses distrust towards the brand",
    "This comment is neutral"
]

/home/maximilian.laechelin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def classify_comment(text):
    result = classifier(text, labels)
    return result['labels'][0]

In [9]:
pkl_files = [
    #'/home/maximilian.laechelin/pwcProject/data/audi_analyzed.pkl',
    #'data/bmw_analyzed.pkl',
    'data/volkswagen_analyzed.pkl',
    #'data/porsche_analyzed.pkl',
]

In [10]:
dataframes = [pd.read_pickle(file) for file in pkl_files]

# Iterate over the DataFrames and print size and length information
for i, df in enumerate(dataframes):
    print(f"DataFrame {i+1}:")
    print(f"  File Path: {pkl_files[i]}")
    print(f"  Number of Rows: {len(df)}")
    print(f"  Number of Columns: {df.shape[1]}")
    print(f"  DataFrame Size: {df.memory_usage(deep=True).sum()} bytes\n")

DataFrame 1:
  File Path: data/volkswagen_analyzed.pkl
  Number of Rows: 5136
  Number of Columns: 6
  DataFrame Size: 2034566 bytes



In [11]:
for file in pkl_files:
    
    df = pd.read_pickle(file)

    tqdm.pandas(desc=f'Processing {file}')
    df['trust_classification'] = df['comment_text'].progress_apply(classify_comment)

    classified_file = file.replace('.pkl', '_classified.pkl')
    df.to_pickle(classified_file)

    # Display the DataFrame in the notebook with all text fully visible
    #pd.set_option('display.max_colwidth', None)
    display(df[['comment_text', 'trust_classification']])


Processing data/volkswagen_analyzed.pkl: 100%|██████████| 5136/5136 [25:59<00:00,  3.29it/s]  


,comment_text,trust_classification
0,Ihr habt es immer noch nicht gescheckt! Bringt...,This comment expresses distrust towards the brand
1,Good Job Volkswagen.,This comment expresses trust in the brand
2,Good one Volkswagen I really like it.,This comment expresses trust in the brand
3,nice,This comment is neutral
4,Looks so boring compared to all these new Chin...,This comment expresses distrust towards the brand
...,...,...
5131,😮,This comment expresses trust in the brand
5132,Nossa kombihome Glorithanna que mora no nosso ...,This comment expresses trust in the brand
5133,isso aí,This comment is neutral
5134,❤❤❤,This comment expresses trust in the brand
